In [2]:
import sys
import os
sys.path.append(os.path.abspath(os.path.dirname(os.getcwd())))

from src.utility.h5_serializable_file import H5Serializable
import src.utility.utils as utils 
from uncertainties import ufloat
from src.photometry_data import PhotometryData
from src.utility.run_cfg import ErebusRunConfig
from src.erebus import Erebus
#from src.frame_normalized_pca import perform_fnpca

In [3]:
ErebusRunConfig.save_schema("../src/schema/run_cfg_schema.json")
lhs1478b = ErebusRunConfig.load("./lhs1478b_test.yaml")

In [4]:
erebus = Erebus(lhs1478b, force_clear_cache = False)

Circular orbit: using gaussian prior for t_sec
Visit jw03730012001 wasn't run yet
Circular orbit: using gaussian prior for t_sec
Visit jw03730013001 wasn't run yet


In [5]:
import numpy as np
import matplotlib.pyplot as plt
fit = erebus.results[1]
mcmc = fit.mcmc
initial_guess = np.array([mcmc.params[p].value for p in mcmc.params])[:-1]
print(initial_guess)

t = fit.time
f = fit.raw_flux
f_model = fit.fit_method(t, *initial_guess)
plt.plot(t, f_model, label='model', zorder=3)
plt.plot(t, f, label='data')
plt.legend()
plt.show()

AttributeError: 'Erebus' object has no attribute 'results'

In [ ]:
for i in range(0, 5):
	plt.plot(t, erebus.results[0].eigenvalues[i])
	plt.show()
	plt.imshow(erebus.results[0].eigenvectors[i])
	plt.show()

In [ ]:
erebus.run()

In [ ]:
fit = erebus.results[1]

res = fit.results
res_nominal_values = [res[k].nominal_value for k in res][:-1]
physical_params = res_nominal_values[:9]
systematic_params = res_nominal_values[9:]

t = fit.time
f = fit.raw_flux
f_model = fit.fit_method(t, *res_nominal_values)
plt.plot(t, f_model, label='model', zorder=3)
plt.plot(t, f, linestyle='', marker='.', label='data')
plt.legend()
plt.show()

f_systematic = fit.systematic_model(t, *systematic_params)
plt.plot(t, f_model / f_systematic, label='model', zorder=3)
plt.plot(t, f / f_systematic, linestyle='', marker='.', label='data')
plt.legend()
plt.show()

print(res['fp']*1e6)
